In [2]:
# Import necessary libraries
import json
import random
import nltk
import pickle
import numpy as np
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

In [4]:

# Download necessary NLTK data
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ajmel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ajmel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Load intents file
DATA_PATH = "C:/Users/Ajmel/Desktop/Ai_c/try/baale_mountain.json"
with open(DATA_PATH, "r") as file:
    intents = json.load(file)

In [8]:

# Prepare data
words = []
classes = []
doc_X = []
doc_y = []

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

In [11]:
# Lemmatize and clean words
words = sorted(set(lemmatizer.lemmatize(w.lower()) for w in words if w.isalnum()))
classes = sorted(set(classes))

with open('vocabulary.pkl', 'wb') as f:
    pickle.dump((words, classes), f)

In [12]:
# Create training data
training = []
out_empty = [0] * len(classes)
for idx, doc in enumerate(doc_X):
    bow = [1 if word in nltk.word_tokenize(doc.lower()) else 0 for word in words]
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    training.append([bow, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Define model
model = Sequential([
    Dense(128, input_shape=(len(train_X[0]),), activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(len(train_y[0]), activation="softmax")
])

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=["accuracy"])
print(model.summary())

c:\Users\Ajmel\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        12,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,775 (85.06 KB)

 Trainable params: 21,775 (85.06 KB)

 Non-trainable params: 0 (0.00 B)

None


In [13]:

# Train model
model.fit(x=train_X, y=train_y, epochs=200, verbose=1,validation_split=0.2)

# Save model
MODEL_PATH = "chatbot.h5"
model.save(MODEL_PATH)

# Load trained model
model = load_model(MODEL_PATH)

Epoch 1/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 380ms/step - accuracy: 0.0263 - loss: 2.7764 - val_accuracy: 0.0909 - val_loss: 2.7367
Epoch 2/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - accuracy: 0.3576 - loss: 2.5298 - val_accuracy: 0.0909 - val_loss: 2.7042
Epoch 3/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.3844 - loss: 2.4355 - val_accuracy: 0.0909 - val_loss: 2.6775
Epoch 4/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.3001 - loss: 2.1682 - val_accuracy: 0.0909 - val_loss: 2.6854
Epoch 5/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.2793 - loss: 2.0580 - val_accuracy: 0.1818 - val_loss: 2.7067
Epoch 6/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.3686 - loss: 1.9184 - val_accuracy: 0.1818 - val_loss: 2.7414
Epoch 7/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.3948 - loss: 1.6770 - val_accuracy: 0.2727 - val_loss: 2.7584
Epoch 8/200
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.4896 - loss: 1.4994 - val_accuracy: 0.2727 - val_